<a href="https://colab.research.google.com/github/mbaroja/TFM/blob/code/Agrupacion_jerarquica_Comparacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering, FeatureAgglomeration
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, silhouette_samples


In [3]:
# Carga de los datos a un dataframe y comprobar dimensionalidad
#df=pd.read_csv("https://raw.githubusercontent.com/mbaroja/TFM/data/data_cleaned.csv",delimiter=";")
#df=pd.read_csv("https://raw.githubusercontent.com/mbaroja/TFM/data/data_cleaned_without_avgcolumns.csv",delimiter=";")
df=pd.read_csv("https://raw.githubusercontent.com/mbaroja/TFM/data/data_cleaned_only_avgcolumns.csv",delimiter=";")
df.shape


(133, 29)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133 entries, 0 to 132
Data columns (total 29 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   AREA        133 non-null    object 
 1   EQUIPO      133 non-null    object 
 2   EDAD        133 non-null    object 
 3   GENERO      119 non-null    object 
 4   V_media     133 non-null    float64
 5   SA_media    133 non-null    float64
 6   SI_media    133 non-null    float64
 7   SP_media    133 non-null    float64
 8   SF_media    133 non-null    float64
 9   CL_media    133 non-null    float64
 10  CH_media    133 non-null    float64
 11  CE_media    133 non-null    float64
 12  CB_media    133 non-null    float64
 13  CD_media    133 non-null    float64
 14  CJ_media    133 non-null    float64
 15  CC_media    133 non-null    float64
 16  CI_media    133 non-null    float64
 17  LS_media    133 non-null    float64
 18  LA_media    133 non-null    float64
 19  LV_media    133 non-null    f

**Preprocesado de los datos de entrada**

In [5]:
#Preprocesado de datos ya que los algoritmos requieren datos numéricos para calcular la matriz de afinidad y realizar el clustering

#identificar las columnas que contienen valores de texto en el conjunto de datos
data_text = df.select_dtypes(include='object')
# aplicar la codificación one-hot utilizando el método get_dummies de Pandas para convertir esas columnas en columnas numéricas
data_encoded = pd.get_dummies(data_text)

In [6]:
#una vez que preprocesados los datos, combinar las columnas numéricas procesadas con el resto del conjunto de datos.
data_numeric = df.select_dtypes(exclude='object')
data_processed = pd.concat([data_numeric, data_encoded], axis=1)

In [7]:
data_processed

,V_media,SA_media,SI_media,SP_media,SF_media,CL_media,CH_media,CE_media,CB_media,CD_media,...,EQUIPO_qmsudmubli,EQUIPO_rflidbinih,EDAD_ijawrgtfuz,EDAD_omygfsbzgq,EDAD_pqgwhrmfdu,EDAD_qmsudmubli,EDAD_vjekcogdhn,GENERO_Femenino,GENERO_Masculino,GENERO_No binario
0,4.333333,5.000000,4.666667,3.000000,4.000000,5.000000,4.000000,4.333333,3.666667,5.000000,...,0,0,0,0,1,0,0,0,1,0
1,4.000000,5.000000,5.000000,2.000000,4.666667,5.000000,5.333333,6.000000,4.333333,4.000000,...,0,0,0,0,1,0,0,0,1,0
2,4.666667,4.000000,4.666667,3.000000,4.333333,5.000000,5.000000,5.000000,3.333333,2.333333,...,0,0,0,1,0,0,0,1,0,0
3,5.000000,5.000000,5.000000,4.000000,4.000000,4.666667,4.666667,4.666667,4.666667,3.666667,...,0,0,0,0,1,0,0,0,1,0
4,2.000000,1.000000,4.666667,1.000000,1.000000,1.000000,1.333333,2.000000,4.000000,5.000000,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,4.000000,4.666667,3.333333,2.000000,2.000000,5.000000,4.000000,4.333333,3.000000,2.333333,...,1,0,0,0,0,1,0,0,0,0
129,3.000000,4.333333,4.333333,1.333333,2.000000,4.666667,2.000000,3.333333,3.333333,4.333333,...,1,0,0,0,0,1,0,0,0,0
130,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,4.666667,3.000000,3.666667,3.000000,3.000000,...,1,0,0,0,0,1,0,0,0,0
131,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,1,0,0,0,0,1,0,0,0,0


In [8]:
X = data_processed.values

**Comparar diferentes parámetros y números de clúster para el algoritmo AgglomerativeClustering**

In [14]:
metrics = ['euclidean', 'l1', 'l2', 'manhattan', 'cosine']
linkages = ['ward', 'complete', 'average', 'single']
resultados = []


for metric in metrics:
  for linkage in linkages:
    if linkage =='ward' and metric != 'euclidean' :
      continue;
    else:
      for n_clusters in range(4,11):

        # Crea una instancia del algoritmo de agrupación jerárquica AgglomerativeClustering
        modelAC = AgglomerativeClustering(n_clusters=n_clusters, metric=metric, linkage=linkage)
        clustersAC=modelAC.fit_predict(X)

        etiquetas = modelAC.labels_

        # Calcular el índice de silueta
        silhouette_avg = silhouette_score(data_processed, etiquetas)

        resultados.append({"Metrica":metric,"Linkage":linkage,"N_clusters":n_clusters,"Silhouette_medio":silhouette_avg})
        print(f"Metrica:{metric}, Linkage:{linkage}, N_clusters:{n_clusters} - Índice de Silhouette Promedio: {silhouette_avg}")



Metrica:euclidean, Linkage:ward, N_clusters:4 - Índice de Silhouette Promedio: 0.34321334351610405
Metrica:euclidean, Linkage:ward, N_clusters:5 - Índice de Silhouette Promedio: 0.17092908264796056
Metrica:euclidean, Linkage:ward, N_clusters:6 - Índice de Silhouette Promedio: 0.1707988768308848
Metrica:euclidean, Linkage:ward, N_clusters:7 - Índice de Silhouette Promedio: 0.17358687754217325
Metrica:euclidean, Linkage:ward, N_clusters:8 - Índice de Silhouette Promedio: 0.1793994139137019
Metrica:euclidean, Linkage:ward, N_clusters:9 - Índice de Silhouette Promedio: 0.18724621760293553
Metrica:euclidean, Linkage:ward, N_clusters:10 - Índice de Silhouette Promedio: 0.17390148592031943
Metrica:euclidean, Linkage:complete, N_clusters:4 - Índice de Silhouette Promedio: 0.3297167901017061
Metrica:euclidean, Linkage:complete, N_clusters:5 - Índice de Silhouette Promedio: 0.2775375183484374
Metrica:euclidean, Linkage:complete, N_clusters:6 - Índice de Silhouette Promedio: 0.280366552850342
Met

In [15]:
df_resultados=pd.DataFrame(resultados)
print(df_resultados)

       Metrica Linkage  N_clusters  Silhouette_medio
0    euclidean    ward           4          0.343213
1    euclidean    ward           5          0.170929
2    euclidean    ward           6          0.170799
3    euclidean    ward           7          0.173587
4    euclidean    ward           8          0.179399
..         ...     ...         ...               ...
107     cosine  single           6          0.417097
108     cosine  single           7          0.408500
109     cosine  single           8          0.396987
110     cosine  single           9          0.338705
111     cosine  single          10          0.288975

[112 rows x 4 columns]


In [18]:
df_resultados.to_csv("resultados_comparacion_agrupacionjerarquica.csv",sep=";",index=False)

In [ ]:
# Calcular el índice de silueta para cada muestra
#silhouette_values = silhouette_samples(data_processed, etiquetas)

# Calcular el índice de silueta individual promedio para cada clúster
#cluster_indices = sorted(set(etiquetas))
#for cluster_idx in cluster_indices:
    # Obtener las muestras pertenecientes al clúster actual
#    cluster_samples = silhouette_values[etiquetas == cluster_idx]
#    cluster_avg_silhouette = cluster_samples.mean()
#    print(f"Índice de Silueta del Clúster {cluster_idx}: {cluster_avg_silhouette}")

In [ ]:
#modelFA=FeatureAgglomeration(n_clusters=5)